In [3]:
import os

Update the Entity

In [4]:
%pwd

'd:\\E2EML\\E2E-ML\\research'

In [5]:
os.chdir("..")

In [6]:
%pwd

'd:\\E2EML\\E2E-ML'

Entity: return type of a function

In [9]:
from dataclasses import dataclass
from pathlib import Path

In [11]:

@dataclass(frozen=True)
class DataIngestionConfig:
    # config.yaml return types
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


Update the configuaration

In [14]:
from iplPredictor.constants import *
from iplPredictor.utils.common import read_yaml, create_directories

In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

Update the components

In [20]:
import os
import urllib.request as request
import zipfile
from iplPredictor import logger
from iplPredictor.utils.common import get_size

In [23]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_zip_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

Update the Pipeline

In [24]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_zip_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-01-04 00:09:18,453: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-04 00:09:18,469: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-04 00:09:18,486: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-04 00:09:18,550: INFO: common: created directory at: artifacts]
[2025-01-04 00:09:18,550: INFO: common: created directory at: artifacts/data_ingestion]
[2025-01-04 00:09:20,303: INFO: 1103171077: artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 1345233
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "3d17052d9ce555f3b512ff6ef453192fad9ae7374df86cd59ce5e845aa91407b"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 254D:08AC:7EF4CA:B96BDF:67782ED8
Accept-Ranges: bytes
Date: Fri, 0